# Non-Federated Logistic Regression on the Synthea MCODE Breast Cancer Dataset
This notebook gives a simple logistic regression classification task over the described Synthea dataset. The notebook assumes that you have followed the instructions in the example's README.md. This should involve setting up the rego_development_playground, Katsu GraphQL interface, and ingesting the synthea MCODE dataset locally.

## Fetching the Data from our GraphQL Server
We first fetch the data.

In [1]:
import requests
import json
query = """
query{
  katsuDataModels
  {
    mcodeDataModels
    {
      mcodePackets{
        subject {
          dateOfBirth
          sex
        }
        cancerCondition {
          dateOfDiagnosis
        }
        cancerRelatedProcedures {
          code {
            label
          }
        }
        cancerDiseaseStatus {
          label
        }
        medicationStatement {
          medicationCode {
            label
          }
        }
      }
    }
  }
}
"""
url = "http://localhost:7999/graphql"
req = requests.post(url, json={'query': query})

In [2]:
print(req.status_code)
all_results = json.loads(req.text)['data']['katsuDataModels']['mcodeDataModels']['mcodePackets'] # what we care about

200


In [3]:
print(len(all_results)) # this number should read ~1884 assuming you have ingested the entire synthea dataset.

1884


## Data Cleaning
Here we drop empty columns, adjust null values, or cut rows.

In [4]:
import pandas as pd
df = pd.json_normalize(all_results) # converts our JSON list into a normalized pandas dataframe

In [5]:
for col in df:
    if df[col].astype(str).nunique() == 1:
        print(col)
        print(df[col].astype(str).unique()) # we drop null-valued columns.
        df = df.drop(col, axis=1)

cancerDiseaseStatus
['nan']


In [6]:
df

,cancerCondition,cancerRelatedProcedures,medicationStatement,subject.dateOfBirth,subject.sex,cancerDiseaseStatus.label
0,[{'dateOfDiagnosis': '1997-12-01T11:12:45Z'}],[],[],1931-12-18,FEMALE,NaN
1,[{'dateOfDiagnosis': '1990-02-11T21:31:03Z'}],[],[],1979-02-14,FEMALE,NaN
2,[{'dateOfDiagnosis': '2020-01-01T01:58:58Z'}],[{'code': {'label': 'Megavoltage radiation the...,[{'medicationCode': {'label': 'tamoxifen citra...,1963-01-14,FEMALE,Patient's condition improved
3,[{'dateOfDiagnosis': '2019-07-13T09:21:07Z'}],[{'code': {'label': 'Megavoltage radiation the...,[{'medicationCode': {'label': '5 ML fulvestran...,1978-07-23,FEMALE,Patient's condition improved
4,[{'dateOfDiagnosis': '2006-06-08T19:33:05Z'}],[],[],1944-06-23,FEMALE,NaN
...,...,...,...,...,...,...
1879,[{'dateOfDiagnosis': '1973-02-14T19:55:20Z'}],[],[],1966-02-16,FEMALE,NaN
1880,[{'dateOfDiagnosis': '2016-12-12T01:48:32Z'}],[{'code': {'label': 'Megavoltage radiation the...,[{'medicationCode': {'label': 'letrozole 2.5 M...,1959-12-26,FEMALE,Patient's condition improved
1881,[{'dateOfDiagnosis': '2018-04-01T13:12:00Z'}],[{'code': {'label': 'Megavoltage radiation the...,[{'medicationCode': {'label': 'palbociclib 100...,1970-04-13,FEMALE,Patient's condition improved
1882,[{'dateOfDiagnosis': '1970-10-09T10:50:30Z'}],[],[],1961-10-11,FEMALE,NaN


In [7]:
df = df.dropna(subset=['cancerDiseaseStatus.label']) # drop any rows that have empty disease status labels.

### Enumerate Cancer_Related_Procedures into Independent Rows

In [8]:
all_procs = set()
for _, row in df.iterrows():
    for i in row['cancerRelatedProcedures']:
        all_procs.add(i['code']['label'])
        
dict_list_procs = []
for _, row in df.iterrows():
    row_dict = dict.fromkeys(all_procs, 0)
    for i in row['cancerRelatedProcedures']:
        row_dict[i['code']['label']] += 1
    dict_list_procs.append(row_dict)
df_procs = pd.DataFrame(dict_list_procs)
df_procs

,Excision of axillary lymph node (procedure),Brachytherapy of breast (procedure),Partial mastectomy (procedure),Partial resection of colon,Excision of sentinel lymph node (procedure),Sentinel lymph node biopsy (procedure),Excision of breast tissue (procedure),Interstitial brachytherapy (procedure),Intracavitary brachytherapy (procedure),Megavoltage radiation therapy using photons (procedure),Radiation oncology AND/OR radiotherapy (procedure)
0,0,0,1,0,0,0,0,0,0,34,0
1,0,0,1,0,0,0,0,0,0,34,0
2,0,0,0,0,0,0,1,0,0,34,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1437,0,0,1,0,0,0,0,0,0,0,0
1438,0,0,0,0,0,0,1,0,0,34,0
1439,0,0,1,0,0,0,0,0,0,34,0
1440,0,0,0,0,0,0,1,0,0,34,0


### Enumerate Medication_Statement into Independent Rows

In [9]:
all_meds = set()
for _, row in df.iterrows():
    for i in row['medicationStatement']:
        all_meds.add(i['medicationCode']['label'])
        
dict_list_meds = []
for _, row in df.iterrows():
    row_dict = dict.fromkeys(all_meds, 0)
    for i in row['medicationStatement']:
        row_dict[i['medicationCode']['label']] += 1
    dict_list_meds.append(row_dict)
df_meds = pd.DataFrame(dict_list_meds)
df_meds

,epirubicin HCl 200 MG in 100 ML Injection,lapatinib 250 MG Oral Tablet,10 ML Fluorouracil 50 MG/ML Injection,ado-trastuzumab emtansine 100 MG Injection,neratinib 40 MG Oral Tablet,cyclophosphamide 1 GM Injection,Carboplatin 10 MG/ML Injectable Solution,10 ML Doxorubicin Hydrochloride 2 MG/ML Injection,PACLitaxel 100 MG Injection,ribociclib 200 MG Oral Tablet,...,10 ML Pamidronate Disodium 3 MG/ML Injection,anastrozole 1 MG Oral Tablet,tamoxifen citrate 10 MG Oral Tablet,5 ML hyaluronidase-oysk 2000 UNT/ML / trastuzumab 120 MG/ML Injection,DOXOrubicin hydrochloride 20 MG per 10 ML Injection,palbociclib 100 MG Oral Capsule,5 ML fulvestrant 50 MG/ML Prefilled Syringe,Verzenio 100 MG Oral Tablet,100 ML Epirubicin Hydrochloride 2 MG/ML Injection,Paclitaxel 100 MG Injection
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,8,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,0,0,0,0,0,8,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1438,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1439,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1440,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


### Parse Diagnosis Age

In [10]:
import datetime
def parse_diagnosis_age(row) -> float:
    """
    A function that returns the difference (in hours) between the diagnosis date and born date of a dataframe entry.
    
    Input: A (Katsu returned) JSON object of the MCODE data.
    Output: The difference between the diagnosis date and born date.
    """
    diag_date = row['cancerCondition'][0]['dateOfDiagnosis']
    diag_age = datetime.datetime(int(diag_date[0:4]), int(diag_date[5:7]), int(diag_date[8:10]))
    born_date = row['subject.dateOfBirth']
    born_age = datetime.datetime(int(born_date[0:4]), int(born_date[5:7]), int(born_date[8:10]))
    difference = diag_age - born_age
    diff_in_hrs = divmod(difference.total_seconds(), 3600)[0] # rounded down
    return diff_in_hrs


In [11]:
diag_age = df.apply(lambda row: parse_diagnosis_age(row), axis=1)
diag_age_rename = diag_age.rename("diagnosisAge")
df = df.join(pd.DataFrame(diag_age_rename))

### Drop Cancer Condition
This probably wouldn't be done in a real workflow with the Synthea MCODE dataset, but I personally cannot parse what, if any of this, is relevant, so I just decided to drop the column since they all have breast cancer.

I also drop the medication_statement and cancer_related_procedures since we've parsed information from them already.

In [12]:
df = df.drop(axis=1, labels=['cancerCondition', 'medicationStatement', 'cancerRelatedProcedures'])

In [13]:
dfnew = pd.concat([df.reset_index(), pd.DataFrame(dict_list_procs), pd.DataFrame(dict_list_meds)], axis=1, ignore_index=False)

### One Hot Encode Cancer_Disease_Status.Label

In [14]:
one_hot = pd.get_dummies(dfnew['cancerDiseaseStatus.label'])
dfnew = dfnew.drop('cancerDiseaseStatus.label', axis=1)
dfnew = dfnew.join(one_hot["Patient's condition improved"])

### Drop Extraneous Columns
We drop any columns that deliver meta-information or information that is already provided by other columns.

In [15]:
dfnew = dfnew.drop(['subject.dateOfBirth', 'index'], axis=1)

## Undersampling the Majority Class
As is pretty clear, we have 1381 data points where the patient's condition improves, with only 61 where they don't. While this is great for the patient, this is not well-balanced data for training a naive binary classifier. Conventional accuracy metrics will not suffice in providing a good understanding of whether the classifier is actually effective, and conventional classifiers will naively optimize for accuracy. Thus, we attempt to balance the dataset by massively reducing the number of data points in our training data. 

By randomly sampling 61 of these 1381 data points, we balance the distribution of data points among each class. We then train a number of classical ML algorithms.

In [16]:
positive_entries = dfnew[dfnew["Patient's condition improved"] == 1]
positive_sample = positive_entries.sample(n=61, random_state=1729)
positive_sample

,diagnosisAge,Excision of axillary lymph node (procedure),Brachytherapy of breast (procedure),Partial mastectomy (procedure),Partial resection of colon,Excision of sentinel lymph node (procedure),Sentinel lymph node biopsy (procedure),Excision of breast tissue (procedure),Interstitial brachytherapy (procedure),Intracavitary brachytherapy (procedure),...,anastrozole 1 MG Oral Tablet,tamoxifen citrate 10 MG Oral Tablet,5 ML hyaluronidase-oysk 2000 UNT/ML / trastuzumab 120 MG/ML Injection,DOXOrubicin hydrochloride 20 MG per 10 ML Injection,palbociclib 100 MG Oral Capsule,5 ML fulvestrant 50 MG/ML Prefilled Syringe,Verzenio 100 MG Oral Tablet,100 ML Epirubicin Hydrochloride 2 MG/ML Injection,Paclitaxel 100 MG Injection,Patient's condition improved
837,473040.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
1439,499344.0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1056,105120.0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1116,586920.0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
347,464280.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,525600.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
823,525600.0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
276,376704.0,1,0,0,0,0,0,1,0,0,...,0,0,0,8,0,0,0,0,0,1
1099,481800.0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [17]:
negative_entries = dfnew[dfnew["Patient's condition improved"] == 0]
negative_entries

,diagnosisAge,Excision of axillary lymph node (procedure),Brachytherapy of breast (procedure),Partial mastectomy (procedure),Partial resection of colon,Excision of sentinel lymph node (procedure),Sentinel lymph node biopsy (procedure),Excision of breast tissue (procedure),Interstitial brachytherapy (procedure),Intracavitary brachytherapy (procedure),...,anastrozole 1 MG Oral Tablet,tamoxifen citrate 10 MG Oral Tablet,5 ML hyaluronidase-oysk 2000 UNT/ML / trastuzumab 120 MG/ML Injection,DOXOrubicin hydrochloride 20 MG per 10 ML Injection,palbociclib 100 MG Oral Capsule,5 ML fulvestrant 50 MG/ML Prefilled Syringe,Verzenio 100 MG Oral Tablet,100 ML Epirubicin Hydrochloride 2 MG/ML Injection,Paclitaxel 100 MG Injection,Patient's condition improved
55,201480.0,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,8,0
62,411744.0,0,0,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
69,473064.0,1,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
73,341640.0,0,0,1,0,1,1,0,0,0,...,0,1,0,8,0,0,0,8,0,0
111,385440.0,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,446784.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1309,394200.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,8,0
1316,621960.0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,8,8,0
1345,595680.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,8,0,0


In [18]:
ml_sample = positive_sample.append(negative_entries)
X = ml_sample.drop("Patient's condition improved", axis=1)
y = ml_sample["Patient's condition improved"]

Since we don't have many points to train with, we don't split into validation sets as well.

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

## Creating a Model Pipeline
Since we have many dimensions relative to our training data volume, we use principal component analysis (PCA) to reduce the number of dimensions in our training data. This will also be used when evaluating test points, and so is part of our model as the first technique in the model 'pipeline'. 

We greatly. reduce the number of dimensions by enforcing that our PCA must finish with 10 dimensions from our dataframe's original 37. Also, we employ the use of PCA whitening to maintain non-correlation between our post-PCA input dimensions.

For our logistic regression, we allow the optimizer to iterate 10000 times to converge. We set a low tolerance (default is 1 for a logistic regression) to ensure strict stopping criteria close to a minimum, at the cost of training speed. We set the solver to 'liblinear' since scikit-learn says that it's suitable for low-volume training data. Since we have little training data, we also set C (the regularization term) to a very low number. This *increases* the regularization parameter's influence, which minimizes overfitting.

In [20]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pca = PCA(n_components=10, whiten=True)
logistic = LogisticRegression(max_iter=10000, tol=0.01, solver='liblinear', random_state=1729, C=0.1, verbose=1)
pipe = Pipeline(steps=[("pca", pca), ("logistic", logistic)])

Finally, we train our model on the data.

In [21]:
pipe.fit(X_train, y_train)

[LibLinear]iter  1 act 1.138e+00 pre 1.098e+00 delta 8.344e-01 f 6.724e+00 |g| 2.732e+00 CG   1
iter  2 act 7.397e-03 pre 7.305e-03 delta 8.344e-01 f 5.585e+00 |g| 1.983e-01 CG   1


Pipeline(steps=[('pca', PCA(n_components=10, whiten=True)),
                ('logistic',
                 LogisticRegression(C=0.1, max_iter=10000, random_state=1729,
                                    solver='liblinear', tol=0.01, verbose=1))])

## Evaluating the Model
Since our sample has been artificially balanced such that positive and negative labels exist in a 1:1 ratio, accuracy is a genuinely good metric for predicting effectiveness. Then, evaluating our model is as simple as calling

In [25]:
pipe.score(X_test, y_test)

0.64

We also evaluate on the AUC score of our model, which is the area under a ROC curve. A value of 0.5 is equivalent to a coin toss. Values closer to 1 are better.

In [26]:
from sklearn.metrics import roc_auc_score
y_pred = pipe.predict(X_test)
roc_auc_score(y_test, y_pred)

0.65

Unfortunately this is still poor discrimination by our classifier; far better than a coin toss, but still not great. Provided more *well-distributed* data, this number could certainly see substantial increases. Hyperparameter searching could also provide benefit, but at such little data this was not tested.